Make use of text in the item names.  Ths notebook does not adjust for seasonality.

In [1]:
import pandas as pd
import os

from utilities import series_to_supervised

# Option 1: For local machine

In [2]:
df_sales = pd.read_csv('./data/beach_restaurant_demand_forecast/sales_train.csv')
df_items = pd.read_csv('./data/beach_restaurant_demand_forecast/items.csv')
df_restaurant = pd.read_csv('./data/beach_restaurant_demand_forecast/restaurants.csv')
df_sales.date = pd.to_datetime(df_sales.date, errors='coerce')

# Option 2: For Google Colab
First download data from 
https://www.kaggle.com/datasets/jeffheaton/demand-forecasting-with-tabular-textual-images

In [3]:
# from google.colab import files
# uploaded = files.upload()

In [4]:
# import io
# df_sales = pd.read_csv(io.BytesIO(uploaded['sales_train.csv']))
# df_items = pd.read_csv(io.BytesIO(uploaded['items.csv']))
# df_restaurant = pd.read_csv(io.BytesIO(uploaded['resturants.csv']))

# df_sales.date = pd.to_datetime(df_sales.date, errors='coerce')

# Option 3: For kaggle notebooks

In [5]:
# PATH_CSV = '/kaggle/input/demand-forecasting-with-tabular-textual-images/beach_demand_forecast'

# df_sales = pd.read_csv(os.path.join(PATH_CSV,"sales_train.csv"))
# df_items = pd.read_csv(os.path.join(PATH_CSV,"items.csv"))
# df_restaurant = pd.read_csv(os.path.join(PATH_CSV,"resturants.csv"))

# df_sales.date = pd.to_datetime(df_sales.date, errors='coerce')

## Explore data

In [6]:
df_items

,id,store_id,name,kcal,cost
0,1,4,Chocolate Cake,554,6.71
1,2,4,Breaded Fish with Vegetables Meal,772,15.09
2,3,1,Sweet Fruity Cake,931,29.22
3,4,1,Amazing Steak Dinner with Rolls,763,26.42
4,5,5,Milk Cake,583,6.07
...,...,...,...,...,...
95,96,5,Blue Ribbon Fish with Bread Lunch,708,21.93
96,97,4,Original Fruity Cod with Bread and Vegetables ...,744,28.65
97,98,2,Original Sweet Milky Soft Drink,579,5.00
98,99,4,Frozen Tomato Soft Drink,423,5.32


In [7]:
df_restaurant

,id,name
0,1,Bob's Diner
1,2,Beachfront Bar
2,3,Sweet Shack
3,4,Fou Cher
4,5,Corner Cafe
5,6,Surfs Up


In [8]:
df_sales

,date,item_id,price,item_count
0,2019-01-01,3,29.22,2.0
1,2019-01-01,4,26.42,22.0
2,2019-01-01,12,4.87,7.0
3,2019-01-01,13,4.18,12.0
4,2019-01-01,16,3.21,136.0
...,...,...,...,...
109595,2021-12-31,96,21.93,0.0
109596,2021-12-31,97,28.65,0.0
109597,2021-12-31,98,5.00,0.0
109598,2021-12-31,99,5.32,0.0


## Load the Glove Embeddings

In [9]:
# !wget -c "https://nlp.stanford.edu/data/glove.6B.zip"
# !unzip glove.6B.zip

In [10]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [11]:
glove_file = './glove.6B/glove.6B.300d.txt'
tmp_file = get_tmpfile("test_word2vec.txt")
_ = glove2word2vec(glove_file, tmp_file)
w2vec_model = KeyedVectors.load_word2vec_format(tmp_file)

/var/folders/nh/pknstycx55zf284hc5q5g2s8p2t_q7/T/ipykernel_47956/806106838.py:3: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  _ = glove2word2vec(glove_file, tmp_file)


# NLP Demo

In [12]:
w2vec_model.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.6713277101516724),
 ('princess', 0.5432624220848083),
 ('throne', 0.5386103987693787),
 ('monarch', 0.5347574949264526),
 ('daughter', 0.4980250597000122),
 ('mother', 0.4956442713737488),
 ('elizabeth', 0.483265221118927),
 ('kingdom', 0.47747087478637695),
 ('prince', 0.4668240249156952),
 ('wife', 0.46473273634910583)]

In [13]:
# for each word in the dictionary of the words there is a vector presentation. The vector is 300-dimensions as stated in `'glove.6B.300d.txt'`
w2vec_model['dog']

array([-1.1043e-01,  8.1217e-01,  7.3668e-02,  1.9023e-01, -5.2888e-02,
        6.1468e-02,  1.6076e-01,  4.1302e-01, -3.0199e-01, -9.0827e-01,
        2.7504e-01, -3.1890e-02, -2.8842e-01,  2.3447e-01,  4.7679e-01,
        5.0124e-01,  2.9371e-01,  2.7029e-01,  5.4745e-02,  9.8038e-02,
        5.7116e-01,  3.6755e-01,  4.0734e-02,  3.4347e-01, -1.8256e-01,
       -2.8935e-01,  2.3826e-02, -1.9401e-01,  2.4444e-01,  1.3407e-01,
       -1.6494e-01, -2.6983e-01, -2.6234e-01, -2.1779e-01, -8.7528e-01,
        7.3822e-01, -8.7931e-02, -1.0876e-02, -2.6540e-01,  3.4668e-01,
       -5.5814e-01,  1.7591e-01,  1.6926e-01, -1.5725e-01, -5.0430e-01,
       -2.0100e-01,  6.6701e-01, -3.2518e-02,  4.5012e-02,  6.5675e-02,
       -1.6061e-01, -7.3363e-01,  2.4642e-01,  3.4325e-01,  2.1899e-01,
        4.8646e-02, -5.9987e-01, -5.8153e-02, -5.1694e-02, -5.7846e-01,
        3.0000e-01,  3.5078e-01,  4.6646e-01, -7.5309e-03,  1.0455e-01,
       -5.1016e-01, -5.5987e-02, -1.0295e-01, -2.6476e-01, -4.12

## Perform NLP Prediction

In [14]:
def process_title(model, name: str):
    """for the given `name` (which is a string of words) calculate the vector representation. 
    The vector representation is the average of all vector representations of words in `name`. 
    Where are the vectors? In the `model`.
    
    EXAMPLE:
        name = "Chocolate Cake"
        get vector repr. for `Chocolate` and `Cake`. Then, average the vectors. 
    """
    v = None
    i = 0
    for word in name.split(' '):
        word = word.lower()
        if word == 'vegi': word = "vegetable"
        if word == 'smoothy': word = "malt"
        i+=1
        if v is None and word in model:
            v=model[word].copy()
        elif word in model:
            v+=model[word]
    v/=i
    return v

In [15]:
process_title(w2vec_model, "Chocolate cake")

array([-0.0758825 ,  0.397085  , -0.008343  , -0.097215  , -0.56650496,
        0.11932001, -0.016513  , -0.23697   ,  0.29838   , -0.396955  ,
        0.2197465 , -0.86015   , -0.242585  ,  0.40074998, -0.47479498,
       -0.43400502,  0.066375  , -0.0704385 , -0.12736   ,  0.14479   ,
        0.30041   ,  0.152275  ,  0.081575  ,  0.561785  , -0.543175  ,
       -0.1077285 , -0.481505  ,  0.01428001, -0.293545  , -0.78625   ,
       -0.45320898,  0.5007    , -0.310665  , -0.33092   , -0.745745  ,
        0.799985  , -0.252535  ,  0.52907   , -0.2017775 ,  0.009743  ,
       -0.5865    , -0.05154   ,  0.15520845,  0.242735  ,  0.103693  ,
       -0.38421   ,  0.43168998, -0.393225  ,  0.126563  ,  0.266575  ,
        0.21884501,  0.019948  ,  0.319965  ,  0.388915  , -0.52665496,
       -0.16352999, -0.313815  ,  0.1312875 ,  0.741905  ,  0.07834   ,
        0.153882  , -0.0286585 ,  0.3736    , -0.19812521,  0.004449  ,
       -0.137377  ,  0.2927    ,  0.21911499, -0.56374   , -0.03

In [16]:
item_lookup = {}
for i, name in zip(list(df_items.id), list(df_items.name)):
    v = process_title(w2vec_model, name)
    item_lookup[i] = v

#r = process_title(w2vec_model, 'breaded fish with vegetables meal')
#print(r)
print(len(item_lookup))

100


Join the items and sales tables so that we can look up the store id for each item.

In [47]:
df_items2 = df_items[['id','store_id']]
df_train = df_sales.merge(df_items2,left_on='item_id',right_on='id')
df_train[['date','item_id','item_count','store_id']]

df_train = df_train.sort_values('date').groupby(['item_id', 'store_id', 'date'], as_index=False)
df_train = df_train.agg({'item_count':['mean']})
df_train.columns = ['item', 'store', 'date', 'sales']
df_train.head()

,item,store,date,sales
0,1,4,2019-01-01,0.0
1,1,4,2019-01-02,0.0
2,1,4,2019-01-03,0.0
3,1,4,2019-01-04,0.0
4,1,4,2019-01-05,0.0


In [48]:
df_train['dow'] = df_train['date'].dt.dayofweek
df_train['doy'] = df_train['date'].dt.dayofyear
df_train

,item,store,date,sales,dow,doy
0,1,4,2019-01-01,0.0,1,1
1,1,4,2019-01-02,0.0,2,2
2,1,4,2019-01-03,0.0,3,3
3,1,4,2019-01-04,0.0,4,4
4,1,4,2019-01-05,0.0,5,5
...,...,...,...,...,...,...
109595,100,2,2021-12-27,0.0,0,361
109596,100,2,2021-12-28,0.0,1,362
109597,100,2,2021-12-29,0.0,2,363
109598,100,2,2021-12-30,0.0,3,364


## Build the sequence data

In [49]:
window = 29 # we use `window` history to predict the future
future_span = 30 # we are predicting this timestamp in the future

In [50]:
df_train.drop('date', axis=1).columns

Index(['item', 'store', 'sales', 'dow', 'doy'], dtype='object')

In [51]:
series2 = series_to_supervised(df_train.drop('date', axis=1), groupby_cols=["store"], value_cols=["item"],
                                window=window, lag=future_span, dropnan=True)
series2.head()

,item(t-29),item(t-28),item(t-27),item(t-26),item(t-25),item(t-24),item(t-23),item(t-22),item(t-21),item(t-20),...,item(t-4),item(t-3),item(t-2),item(t-1),item,store,sales,dow,doy,item(t+30)
29,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1,4,0.0,2,30,1.0
30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1,4,0.0,3,31,1.0
31,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1,4,0.0,4,32,1.0
32,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1,4,0.0,5,33,1.0
33,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1,4,0.0,6,34,1.0


In [52]:
len(series2)

109246

In [53]:
# Utility function to create sequences.
def series_to_supervised2(data: pd.DataFrame, window=1, lag=1, dropnan=True):
    """Use history according to `window` size to predict the future specified in `lag`.
    The output is a wide-format dataframe. This method creates (1, ..., window) features for each of the columns in `data`.
    """
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [54]:
series = series_to_supervised2(df_train.drop('date', axis=1), window=window, lag=future_span, dropnan=False)
series.head()

,item(t-29),store(t-29),sales(t-29),dow(t-29),doy(t-29),item(t-28),store(t-28),sales(t-28),dow(t-28),doy(t-28),...,item(t),store(t),sales(t),dow(t),doy(t),item(t+30),store(t+30),sales(t+30),dow(t+30),doy(t+30)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,4,0.0,1,1,1.0,4.0,0.0,3.0,31.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,4,0.0,2,2,1.0,4.0,0.0,4.0,32.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,4,0.0,3,3,1.0,4.0,0.0,5.0,33.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,4,0.0,4,4,1.0,4.0,0.0,6.0,34.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,4,0.0,5,5,1.0,4.0,0.0,0.0,35.0


In [55]:
len(series)

109600

Remove sequences that did not have enough data.

In [56]:
# Remove edge cases, where there were not enough values to complete a series
last_item = 'item(t-%d)' % window
last_store = 'store(t-%d)' % window
last_dow = 'dow(t-%d)' % window
last_doy = 'doy(t-%d)' % window
series = series[(series['store(t)'] == series[last_store])]
series = series[(series['item(t)'] == series[last_item])]
#series = series[(series['dow(t)'] == series[last_dow])]
#series = series[(series['doy(t)'] == series[last_doy])]

In [57]:
len(series)

106700

We will predict with sales, and our engineered features.

In [58]:
import numpy as np
# from tensorflow.keras.utils import to_categorical

def drop_column(df, col):
    columns_to_drop = [('%s(t+%d)' % (col, future_span))]
    for i in range(window, 0, -1):
        columns_to_drop += [('%s(t-%d)' % (col, i))]
    df.drop(columns_to_drop, axis=1, inplace=True, errors='ignore')
    df.drop([f"{col}(t)"], axis=1, inplace=True, errors='ignore')

# def cat_seq(df, col):
#     return to_categorical(df[col].values)

# Label
labels_col = 'sales(t+%d)' % future_span
labels = series[labels_col]
series.drop(labels_col, axis=1, inplace=True)
series.drop('item(t+%d)' % future_span, axis=1, inplace=True)
series.drop('store(t+%d)' % future_span, axis=1, inplace=True)
series.drop('dow(t+%d)' % future_span, axis=1, inplace=True)
series.drop('doy(t+%d)' % future_span, axis=1, inplace=True)

# Get sales sequences
series3 = series.copy()
drop_column(series3, "item")
drop_column(series3, "store")
drop_column(series3, "dow")
drop_column(series3, "doy")
# sales_series is a 2-D array of shape (len(series), len(window) + future_span + day itself)
sales_series = series3.values

# Day of week as a number
series3 = series.copy()
drop_column(series3, "item")
drop_column(series3, "store")
drop_column(series3, "doy")
drop_column(series3, "sales")
dow_series = series3.values

# Get day of year sequences
series3 = series.copy()
drop_column(series3, "item")
drop_column(series3, "store")
drop_column(series3, "dow")
drop_column(series3, "sales")
doy_series = series3.values

t1 = sales_series.reshape(sales_series.shape + (1,))
t2 = dow_series.reshape(dow_series.shape + (1,)) 
t3 = doy_series.reshape(doy_series.shape + (1,))

# Create predictors (x)
# Create x1, which consists features `sales`, `dow`, `doy`
x1 = np.concatenate([t1,t2,t3],axis=2)
# Create x2, which is the encoding of items according to the word2vec model
vec_size = w2vec_model['test'].shape[0]
lst = []
for item in list(series['item(t-1)']):
    lst.append(item_lookup[item])
x2 = np.concatenate(lst).reshape((series.shape[0],vec_size))

# Combine all features
x = [x1,x2]

In [59]:
labels

29        0.0
30        0.0
31        0.0
32        0.0
33        0.0
         ... 
109595    NaN
109596    NaN
109597    NaN
109598    NaN
109599    NaN
Name: sales(t+30), Length: 106700, dtype: float64

In [60]:
sales_series

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [61]:
print(t1.shape)
print(t2.shape)
print(t3.shape)

(106700, 30, 1)
(106700, 30, 1)
(106700, 30, 1)


Extract the predictors (x sequences) and the label (future prediction)

In [62]:
TEST_SIZE = 0.4

mask = np.random.random(size=x[0].shape[0])<TEST_SIZE

X_train = []
X_valid = []

for subx in x:
    X_train.append(subx[~mask])
    X_valid.append(subx[mask])

Y_train = labels.values[~mask]
Y_valid = labels.values[mask]

print('Train set shape x1:', X_train[0].shape)
print('Train set shape x2:', X_train[1].shape)
print('Validation set shape x1:', X_valid[0].shape)
print('Validation set shape x2:', X_valid[1].shape)

Train set shape x1: (63873, 30, 3)
Train set shape x2: (63873, 300)
Validation set shape x1: (42827, 30, 3)
Validation set shape x2: (42827, 300)


Construct the neural network.

In [63]:
import tensorflow as tf 
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout, concatenate, Input
import keras

epochs = 500
batch = 256
lr = 0.0003
adam = tf.keras.optimizers.Adam(lr)

model = Sequential()

A1 = Input(shape=(X_train[0].shape[1], X_train[0].shape[2]),name='A1')
A2 = Conv1D(filters=64, kernel_size=8, activation='relu')(A1)
A3 = MaxPooling1D(pool_size=2)(A2)
A4 = Flatten()(A3)
A5 = Dense(50, activation='relu')(A4)
A6 = Dropout(0.2)(A5)

B1 = Input(shape=X_train[1].shape[1],name='B1')
B2 = Dense(16, activation='relu',name='B2')(B1)

M1 = concatenate([A6,B2])
M2 = Dense(1,name='M2')(M1)

model = Model(inputs=[A1, B1],outputs=[M2])
model.compile(loss='mse', optimizer=adam)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
A1 (InputLayer)                 [(None, 30, 3)]      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 23, 64)       1600        A1[0][0]                         
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 11, 64)       0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 704)          0           max_pooling1d_1[0][0]            
____________________________________________________________________________________________

Fit the neural network.

In [64]:
from tensorflow.keras.callbacks import EarlyStopping

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, 
        verbose=1, mode='auto', restore_best_weights=True)

cnn_history = model.fit(X_train, Y_train, callbacks=[monitor],
    validation_data=(X_valid, Y_valid), epochs=epochs, verbose=2)

Epoch 1/500
1997/1997 - 3s - loss: nan - val_loss: nan
Epoch 2/500
1997/1997 - 3s - loss: nan - val_loss: nan
Epoch 3/500
1997/1997 - 3s - loss: nan - val_loss: nan
Epoch 4/500
1997/1997 - 3s - loss: nan - val_loss: nan
Epoch 5/500
1997/1997 - 3s - loss: nan - val_loss: nan
Restoring model weights from the end of the best epoch.


TypeError: object of type 'NoneType' has no len()

Predict and evaluate the validation data.

In [46]:
from sklearn.metrics import mean_squared_error
import numpy as np

cnn_train_pred = model.predict(X_train)
cnn_valid_pred = model.predict(X_valid)
print('Train rmse:', np.sqrt(mean_squared_error(Y_train, cnn_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_valid, cnn_valid_pred)))

ValueError: Input contains NaN.

Plot the training curve.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(cnn_history.history['loss'], label='Train loss')
plt.plot(cnn_history.history['val_loss'], label='Validation loss')
fig.legend()
fig.suptitle('CNN')
plt.xlabel("Epochs")
plt.ylabel("MSE")

plt.show()